# 2. SDKを使って複数のCSVファイルをアップロードしてみよう
このチュートリアルでは、**intdash SDK for Python** （以下、intdash SDKと呼びます）を使用して多数のCSVファイルをアップロードする方法を紹介します。
今回のケースでは、CSVファイル1ファイルごとに1つの計測を作成します。CSVファイルのフォーマットについては、以下の「事前準備」を参考にしてください。

## 2.0 事前準備
本シナリオを実施する前に、以下を用意する必要があります。

- データ登録用のエッジ
- アップロードする複数のCSV


### 使用データ
本チュートリアルでは、以下のデータをサーバー側に準備する必要があります。  
本項では以下のデータ名で処理を実施しています。

|データ項目|本シナリオで登場するデータ名|
|:---|:---|
|時系列データを登録するエッジ|edge1|
|時系列データが格納されたCSV| sampleX.csv （X = 任意の数字）|

#### アップロードするCSVデータの詳細

今回アップロードするCSVは、以下の条件を満たすものとします。
 * 第1行は、各列の名称を文字列で格納していること
 * 第1列は、タイムスタンプを格納していること
 * 第2列目以降には、各列名ごとのデータを格納していること

<img src="https://github.com/aptpod/aws-marketplace-tutorials/blob/master/img/img1.png?raw=true\">

第1行に記載されている名称をデータの名前（ `data_id` ）として使用します。

### アップロードするCSVファイルの配置
登録する対象となる時系列データが格納された複数のCSVファイルは、本Jupyter Notebookと同じディレクトリに存在する `csv` ディレクトリ配下に配置します。本チュートリアルでは、既に配置されているサンプルのCSVファイルを用いて実施しています。

### パッケージのimportとクライアントの生成
`intdash.Client` に与える `url` は intdashサーバーの環境情報から、`username` と `password` はログイン用エッジで発行したアクセス情報を指定してください。  

In [59]:
import pandas as pd
import math

import intdash
from intdash import timeutils

# Create client
client = intdash.Client(
    url = "https://example.intdash.jp",
    username = "edge1",
    password="password_here"
)

以上で事前準備は終了です。

## 2.1 データのアップロードに使用するエッジを取得する
まず最初に、CSVファイルをアップロードする際に使用するエッジを取得します。

In [20]:
edges = client.edges.list(name='edge1')
edge1 = edges[0]

In [21]:
edge1.name

'edge1'

## 2.2 CSVファイルを取得する
各CSVファイルを `pandas.DataFrame` として読み込みます。ここでは `csv/` ディレクトリに格納されているCSVファイルを対象にします。

In [22]:
import glob 

csv_files = glob.glob('./csv/*')

dfs = []

for csv_path in csv_files:
    df = pd.read_csv(csv_path, index_col=0).groupby("time").last()
    dfs.append(df)

## 2.3 CSVファイルごとに計測を作成し、データを登録する
ここからは1つのCSVファイルごとの処理に焦点をあてて解説します。これらの処理を繰り返すことで、複数のCSVファイルをアップロードすることができます。  
※ すぐに複数ファイルを処理したい場合、本節をスキップしてください。

In [23]:
# Pick first DataFrame.
df = dfs.pop(0)

データの最初の時刻を計測の開始時刻とし、計測を作成します。

In [24]:
new_measurements = client.measurements.create(
    name='csv_data',  # Define name of measurement.
    basetime=timeutils.str2timestamp(df.index[0]), #  Use timestamp of the first datapoint as the basetime of the measurement.
    edge_uuid=edge1.uuid
)

計測の生成後、DataFrameを `DataPoint` 形式に変換します。

In [25]:
datapoints = []

for data_id, values in df.to_dict().items():
    for time, value in values.items():
        
        if math.isnan(value) or value is '':
            continue
            
        datapoints.append(
            intdash.DataPoint(
                elapsed_time= timeutils.str2timestamp(time) - new_measurements.basetime, # Time elapsed from the start of measurement.
                data_type=intdash.DataType.float,
                channel=1, # fixed at 1.
                data_payload=intdash.data.Float(data_id=data_id, value=value).to_payload()
            )
        )

変換が完了したら、先程作成した計測に時系列データを紐付けます。

In [26]:
client.data_points.store(
    measurement_uuid=new_measurements.uuid,
    data_points=datapoints
)

これで1つのCSVデータのアップロードが完了しました。

## 2.4 複数CSVファイルに対してまとめて登録処理を行う
以下は、前節の処理を1つにまとめ、複数のDataFrameに対して繰り返し処理をおこなっています。

In [27]:
for df in dfs:
    # Create a measurement.
    new_measurements = client.measurements.create(
        name='csv_data',  # Define name of measurement.
        basetime=timeutils.str2timestamp(df.index[0]), #  Use timestamp of the first datapoint as the basetime of the measurement.
        edge_uuid=edge1.uuid
    )
    
    # Convert DataFrames to DataPoints.
    datapoints = []

    for data_id, values in df.to_dict().items():
        for time, value in values.items():
            
            if math.isnan(value) or value is '':
                continue

            datapoints.append(
                intdash.DataPoint(
                    elapsed_time= timeutils.str2timestamp(time) - new_measurements.basetime, # Time elapsed from the start of measurement.
                    data_type=intdash.DataType.float,
                    channel=1, # fixed at 1.
                    data_payload=intdash.data.Float(data_id=data_id, value=value).to_payload()
                )
            )
            
    # Store datapoints.
    client.data_points.store(
        measurement_uuid=new_measurements.uuid,
        data_points=datapoints
    )

上記を実行すると、複数のCSVファイルをアップロードすることができます。
Visual M2M Data Visualizerで[Stored Data]を確認すると、計測が登録されていることが確認できます。
<img src="https://github.com/aptpod/aws-marketplace-tutorials/blob/master/img/img3.png?raw=true\">